<a href="https://colab.research.google.com/github/null-buster/speech-to-text/blob/main/SpeechToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepspeech

     |████████████████████████████████| 9.2MB 3.7MB/s 


In [2]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   652  100   652    0     0   4759      0 --:--:-- --:--:-- --:--:--  4759
100  180M  100  180M    0     0  62.4M      0  0:00:02  0:00:02 --:--:-- 70.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   654  100   654    0     0   4389      0 --:--:-- --:--:-- --:--:--  4389
100  909M  100  909M    0     0  47.6M      0  0:00:19  0:00:19 --:--:-- 44.0M


In [3]:
!sudo apt-get install sox
!pip install sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 12 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main a

In [4]:
!pip install ffmpeg-python

In [5]:
!pip install Resemblyzer


     |████████████████████████████████| 15.7MB 293kB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26310 sha256=9e6d66797a1f0ee9148eeaf3ef30193fdefcb8af595b55e9d51aa78537d28fd6
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71389 sha256=302a758dfa82e3dbd4e9232a86095e2ddc2fe6fe16de72dd2f734e5cfef83ead
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
Successfully built typing webrtcvad


In [6]:
!pip3 install spectralcluster

In [7]:
!pip install natsort

In [8]:
import ffmpeg
import os
def convertToWav(file_path, output_path):
  if not os.path.exists(output_path + '/' + file_path.split('/')[-1].split('.')[-2]+'.wav'):
    audio_input = ffmpeg.input(file_path)
    new_filename = output_path + '/' + file_path.split('/')[-1].split('.')[-2]+'.wav'
    audio_output = ffmpeg.output(audio_input, new_filename)
    ffmpeg.run(audio_output)


In [9]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from spectralcluster import SpectralClusterer
from resemblyzer.audio import sampling_rate

def speakerDiarization(file_path):
  #give the file path to your audio file
  #audio_file_path = '/content/test audio.wav'
  audio_file_path = file_path
  wav_fpath = Path(audio_file_path)

  wav = preprocess_wav(wav_fpath)
  encoder = VoiceEncoder("cpu")
  _, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
  print(cont_embeds.shape)
  clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

  labels = clusterer.predict(cont_embeds)
  print(labels)
  times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
  labelling = []
  start_time = 0

  for i,time in enumerate(times):
    if i>0 and labels[i]!=labels[i-1]:
      temp = [str(labels[i-1]),start_time,time]
      labelling.append(tuple(temp))
      start_time = time
    if i==len(times)-1:
      temp = [str(labels[i]),start_time,time]
      labelling.append(tuple(temp))

  return labelling

In [10]:
import ffmpeg
def cropAudio(filepath, labelling, outputpath):
  audio_input = ffmpeg.input(filepath)
  num = len(labelling)
  for label in labelling:
    if not os.path.exists(outputpath + '/' + filepath.split('/')[-1].split('.')[-2]+ str(num) +'.wav'): 
      audio_crop = audio_input.audio.filter('atrim', start=label[1], end=label[2])
      crop_name = outputpath + '/' + filepath.split('/')[-1].split('.')[-2]+ str(num) +'.wav'
      audio_output = ffmpeg.output(audio_crop, crop_name)
      ffmpeg.run(audio_output)
    num -= 1
    #print(label[0])
    #print(label[1])
    #print(label[2])
    #print(type(label[1]))



In [11]:
#import os
def getInference(pbmm_path, scorer_path, filepath):
  #command = "deepspeech --model deepspeech-0.9.3-models.pbmm --scorer deepspeech-0.9.3-models.scorer --audio /content/TestAudio.wav "
  command = 'deepspeech --model ' + pbmm_path + ' --scorer '+ scorer_path + ' --audio ' + filepath
  stream = os.popen(command)
  output = stream.read()
  print(output)
  return output


  


In [12]:
import natsort
def getDiarized(croppedWavPath, labelling):
  path_to_files = os.listdir(os.path.abspath('.') + '/' + croppedWavPath)
  sorted_names = natsort.natsorted(path_to_files, reverse=True)
  #print(sorted_names)
  num = len(labelling)
  dialogues = []
  #print(getInference('/content/deepspeech-0.9.3-models.pbmm', '/content/deepspeech-0.9.3-models.scorer', '/content/cropped_Wavs/TestAudio1.wav'))
  for names in sorted_names:
    #print(os.path.abspath('.') + '/' + croppedWavPath + '/' + names)
    text = getInference('/content/deepspeech-0.9.3-models.pbmm', '/content/deepspeech-0.9.3-models.scorer', os.path.abspath('.') + '/' + croppedWavPath + '/' + names)
    speaker_label = 'Speaker ' + labelling[num-1][0] + ': '
    #print(speaker_label)
    print(text)
    if text is not None:
      final_text = speaker_label + text
      dialogues.append(final_text)
      print(final_text)
    num -= 1
  print(dialogues)
  return dialogues

In [13]:
audio_folder = '/audio_folder'
if not os.path.exists(os.path.abspath('.') + audio_folder):
  os.mkdir(os.path.abspath('.') + audio_folder)
convertedWav_folder = '/converted_Wavs'
if not os.path.exists(os.path.abspath('.') + convertedWav_folder):
  os.mkdir(os.path.abspath('.') + convertedWav_folder)
croppedWav_folder = '/cropped_Wavs'
if not os.path.exists(os.path.abspath('.') + croppedWav_folder):
  os.mkdir(os.path.abspath('.') + croppedWav_folder)


In [14]:
#continue from here
def writeOutput(audio_file, converted_Wavs, cropped_Wavs):
  convertToWav(audio_file , converted_Wavs)
  labelling = speakerDiarization(converted_Wavs + audio_file.split('/')[-1].split('.')[-2]+'.wav')
  cropAudio(converted_Wavs + audio_file.split('/')[-1].split('.')[-2]+'.wav', labelling, cropped_Wavs)
  
  with open(os.path.abspath('.') + '/' +'Output.txt', 'w' ) as f:
    f.write("Transcript of " + audio_file.split('/')[-1].split('.')[-2]+":\n\n" )
    text = getInference('/content/deepspeech-0.9.3-models.pbmm', '/content/deepspeech-0.9.3-models.scorer', converted_Wavs + audio_file.split('/')[-1].split('.')[-2]+'.wav')
    f.write("Trascript of entire file without diarization is :\n" + text)
    f.write("Transcript with diarization is : \n")
    dialogues = getDiarized(cropped_Wavs.split('/')[-1], labelling)
    for entry in dialogues:
      f.write(entry)
    


  


Output:

After the above cells have run.
The `/content/audio_folder/NewsRecording.mp3` path in the below `writeOutput` function call needs to be changed to the audio uploaded inside the `audio_folder` before running the cell below. The script would run the inference and output the `Output.txt` containing the inference text in the root folder that is `/content/Output.text` in case of Google Collab.



In [ ]:
writeOutput('/content/audio_folder/NewsRecording.mp3', '/content/converted_Wavs/', '/content/cropped_Wavs')

Loaded the voice encoder model on cpu in 0.03 seconds.
(4122, 256)
[0 0 0 ... 1 1 1]
what we have heard from the president expected to find out who is getting his final pardons were more do we know about that now well the present appears to me kind of don a one eighty on who were expecting to be on the list we kinotah it was going to be this and all be all lissome present or he included a lot of his allies on here but we are now learning that the present is not expected to pardon himself or his family members in this list that we are going to get in the next few hours were expecting an interstate because of some new reporting about a meeting that happened at the white house on saturday night where the president was in the west wing he was there with his white house counsel patapon eric cushman who was another white house attorney who you might remember that was someone who was on the senate floor last year defending the president in his impeachment trial and were told that direcshun ab

Code ends here